# Regionprops / napari_simpleitk_image_processing_label_statistics / napari_cupy_image_processing_measurements comparison

Benchmark feedback for Robert Haase

In [1]:
import numpy as np
import zarr
from skimage.measure import regionprops
from skimage.util import map_array
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load raw data

image = zarr.load(r"D:\EHG\Explants\INSL3\C2-EH3919_DMSOEtOH_Explant-2_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.ome_Processed.aivia_Processed.aivia.tif8.zarr")

# Load the segmented image

labels = zarr.load("D:\EHG\Explants\INSL3\Labels\C2-EH3919_DMSOEtOH_Explant-2_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.ome_Processed.aivia_Processed.aivia_cp_masks.tif5.zarr")

In [3]:
import napari

In [4]:
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_labels(labels)

<Labels layer 'labels' at 0x207a31ac9d0>

In [ ]:
import time

for i in range(0, 10):
    start_time = time.time()
    
    # get dictionary of measurements
    props = regionprops(labels, intensity_image=image, cache=True,)
        
    # read out arrays of values
    
    area = [s.area for s in props]
    equivalent_diameter_area = [s.equivalent_diameter_area for s in props]
    intensity_max = [s.intensity_max for s in props]
    intensity_mean = [s.intensity_mean for s in props]
    intensity_min = [s.intensity_min for s in props]
    solidity = [s.solidity for s in props]
        
    print("Determining label statistics using scikit-image took " + str(time.time() - start_time) + " s")

In [ ]:
from napari_simpleitk_image_processing import label_statistics
import pandas as pd

for i in range(0, 10):
    start_time = time.time()
    
    # get dictionary of measurements
    props = label_statistics(image, labels, None, True, True, True, True, True, True)
        
    # read out arrays of values
    content = pd.DataFrame(props)

    intensity_max = content['maximum']
    intensity_mean = content['mean']
    intensity_min = content['minimum']
    principal_moments0 = content['principal_moments0']
        
    print("Determining label statistics using simpleitk took " + str(time.time() - start_time) + " s")

In [ ]:
from napari_cupy_image_processing import measurements

for i in range(0, 10):
    start_time = time.time()
    
    # get dictionary of measurements
    props = measurements(image, labels)
        
    # read out arrays of values
    content = pd.DataFrame(props)

    max_vector = content['maximum']
    mean_vector = content['mean']
    min_vector = content['minimum']
    median = content['median']
        
    print("Determining label statistics using cupy took " + str(time.time() - start_time) + " s")

In [5]:
def measures(
        image_layer : viewer.layers['image'],
        labels_layer: viewer.layers['labels'],
        napari_viewer : napari.Viewer = None,
        size: bool = True,
        intensity: bool = True,
        position: bool = False):
    
    import cupy
    from cupyx.scipy import ndimage

    if image_layer is not None and labels_layer is not None:

        labels = cupy.asarray(labels_layer.data)
        image = cupy.asarray(image_layer.data)

        df = {}

        for l in range(1, labels.max().get() + 1):
            _append_to_column(df, "label", l)
            if position:
                for i, x in enumerate(ndimage.center_of_mass(image, labels, l)):
                    _append_to_column(df, "center_of_mass_" + str(i), x.get())
                for i, x in enumerate(ndimage.minimum_position(image, labels, l)):
                    _append_to_column(df, "minimum_position_" + str(i), x)
                for i, x in enumerate(ndimage.maximum_position(image, labels, l)):
                    _append_to_column(df, "maximum_position_" + str(i), x)

            mean = None
            if intensity:
                x = ndimage.mean(image, labels, l)
                mean = x.get()
                _append_to_column(df, "mean", x.get())
                x = ndimage.minimum(image, labels, l)
                _append_to_column(df, "minimum", x.get())
                x = ndimage.maximum(image, labels, l)
                _append_to_column(df, "maximum", x.get())
                x = ndimage.median(image, labels, l)
                _append_to_column(df, "median", x.get())
                x = ndimage.standard_deviation(image, labels, l)
                _append_to_column(df, "standard_deviation", x.get())

            if size:
                sum_labels = ndimage.sum_labels(image, labels, l).get()
                if mean is None:
                    mean = ndimage.mean(image, labels, l).get()
                pixel_count = sum_labels / mean
                _append_to_column(df, "pixel_count", pixel_count)

        result = {}
        for k, v in df.items():
            result[k] = np.asarray(v).tolist()

        return result
        

def _append_to_column(dictionary, column_name, value):
    if column_name not in dictionary.keys():
        dictionary[column_name] = []
    dictionary[column_name].append(value)

In [ ]:
import time

for i in range(0, 10):
    start_time = time.time()
    
    # get dictionary of measurements
    props = measures(image, labels)
        
    # read out arrays of values
    content = pd.DataFrame(props)

    max_vector = content['maximum']
    mean_vector = content['mean']
    min_vector = content['minimum']
    median = content['median']
        
    print("Determining label statistics using cupy took " + str(time.time() - start_time) + " s")